In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('transactions_train.csv')

In [3]:
data.head()

,step,type,amount,nameOrig,oldbalanceOrig,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0


In [4]:
#vamos a borrar columnas que sean redundantes o no aporten nada al algoritmo por ser unicas

data = data.drop(['nameOrig', 'nameDest'], axis = 1)

In [5]:
data.head()

,step,type,amount,oldbalanceOrig,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud
0,1,PAYMENT,9839.64,170136.0,160296.36,0.0,0.0,0
1,1,PAYMENT,1864.28,21249.0,19384.72,0.0,0.0,0
2,1,TRANSFER,181.00,181.0,0.00,0.0,0.0,1
3,1,CASH_OUT,181.00,181.0,0.00,21182.0,0.0,1
4,1,PAYMENT,11668.14,41554.0,29885.86,0.0,0.0,0


In [6]:
data.shape

(6351193, 8)

In [7]:

df_fraud = data[data['isFraud'] == 1]

df_nofraud = data[data['isFraud'] == 0]

df_nofraud = df_nofraud.head(12000)

data = pd.concat([df_fraud, df_nofraud], axis = 0)

In [8]:
data.head()

,step,type,amount,oldbalanceOrig,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud
2,1,TRANSFER,181.0,181.0,0.0,0.0,0.0,1
3,1,CASH_OUT,181.0,181.0,0.0,21182.0,0.0,1
251,1,TRANSFER,2806.0,2806.0,0.0,0.0,0.0,1
252,1,CASH_OUT,2806.0,2806.0,0.0,26202.0,0.0,1
680,1,TRANSFER,20128.0,20128.0,0.0,0.0,0.0,1


In [9]:
data.shape

(19717, 8)

In [10]:
#se pasa numerico variables categoricas  
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

data['type'] = data['type'].astype('category')

type_encode = LabelEncoder()

data['type'] = type_encode.fit_transform(data.type)

In [11]:
#como esta variable categoria es nominal, hay que hacer otro paso ahora. Si fuera ordinal, ya estaria. 


type_one_hot = OneHotEncoder()

type_one_hot_encode = type_one_hot.fit_transform(data.type.values.reshape(-1,1)).toarray()
 

ohe_variable = pd.DataFrame(type_one_hot_encode, columns = ["type_"+str(int(i)) for i in range(type_one_hot_encode.shape[1])])

data = pd.concat([data, ohe_variable], axis=1)


data = data.drop('type', axis = 1)


data.head()

,step,amount,oldbalanceOrig,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,type_0,type_1,type_2,type_3,type_4
2,1.0,181.0,181.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
3,1.0,181.0,181.0,0.0,21182.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
251,1.0,2806.0,2806.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
252,1.0,2806.0,2806.0,0.0,26202.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
680,1.0,20128.0,20128.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


In [12]:
#Checking every column for missing values

data.isnull().any()

step              True
amount            True
oldbalanceOrig    True
newbalanceOrig    True
oldbalanceDest    True
newbalanceDest    True
isFraud           True
type_0            True
type_1            True
type_2            True
type_3            True
type_4            True
dtype: bool

In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 27353 entries, 2 to 19716
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   step            19717 non-null  float64
 1   amount          19717 non-null  float64
 2   oldbalanceOrig  19717 non-null  float64
 3   newbalanceOrig  19717 non-null  float64
 4   oldbalanceDest  19717 non-null  float64
 5   newbalanceDest  19717 non-null  float64
 6   isFraud         19717 non-null  float64
 7   type_0          19717 non-null  float64
 8   type_1          19717 non-null  float64
 9   type_2          19717 non-null  float64
 10  type_3          19717 non-null  float64
 11  type_4          19717 non-null  float64
dtypes: float64(12)
memory usage: 2.7 MB


In [14]:
#se coloca 0 en valores vacios  pero hay otras maneras

data = data.fillna(0)

In [22]:
data.to_csv('fraud_prediction.csv')


In [15]:
#KNN 


features = data.drop('isFraud', axis = 1).values
target = data['isFraud'].values

In [16]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size = 0.3, random_state = 42, stratify = target)

In [17]:
from sklearn.neighbors import KNeighborsClassifier

#Se inicializa con 3 

knn_classifier = KNeighborsClassifier(n_neighbors=3)

knn_classifier.fit(X_train, y_train)

#Nivel de precision

knn_classifier.score(X_test, y_test)

0.9841579332195954

In [19]:
#ver parametros

import numpy as np
from sklearn.model_selection import GridSearchCV


grid = {'n_neighbors' : np.arange(1, 25)}


knn_classifier = KNeighborsClassifier()

knn = GridSearchCV(knn_classifier, grid, cv = 10)

knn.fit(X_train, y_train)

#mejor parametro
knn.best_params_


{'n_neighbors': 1}

In [20]:
knn.best_score_

0.9833917458550573

In [21]:
#escalamos para tener una mejor precision 
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline


pipeline_order = [('scaler', StandardScaler()), ('knn', KNeighborsClassifier(n_neighbors = 1))]

pipeline = Pipeline(pipeline_order)


knn_classifier_scaled = pipeline.fit(X_train, y_train)


knn_classifier_scaled.score(X_test, y_test)

0.9957348281745064